In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"
from omegaconf import OmegaConf
import numpy as np
import matplotlib.pyplot as plt
import json
import pandas as pd
import pickle
import math

current_path = "/home/giacomo/ecgi"#os.getcwd()
results_path = "/home/giacomo/ecgi/results"

# Plot settings
labelssize = 14
ticksize = 12
legendsize = 12
colors = ["#fec44f", "#d95f0e", "#fff7bc"]#["#66c2a5", "#fc8d62", "#8da0cb"]

# FMM settings 
num_features = 260
num_waves = 2

def get_subfolders_at_depth(folder_path, depth):
    #TODO: DOES NOT WORK FOR DEPTH DIFFERENT THAN 1
    assert depth==1
    # Source: https://stackoverflow.com/questions/7159607/list-directories-with-a-specified-depth-in-python
    path = folder_path
    path = os.path.normpath(path)
    res = []
    for root,dirs,files in os.walk(path, topdown=True):
        cuurent_depth = root[len(path) + len(os.path.sep)-1:].count(os.path.sep)
        if cuurent_depth == depth:
            # We're currently two directories in, so all subdirs have depth 3
            res += [os.path.join(root, d) for d in dirs]
            # dirs[:] = [] # Don't recurse any deeper
    # res.pop(0)
    return res

def get_conf_from_paths(paths):
    conf_list = []
    for p in paths:
        file_path = os.path.join(p,"conf.yaml")
        # with open(file_path,"r") as fp:
        cfg = OmegaConf.load(file_path)
        cfg["path"] = p
        conf_list.append(cfg)
    return conf_list

def get_completed_conf_files(config_files, log=True):
    completed_config_files = []
    not_completed_config_files = []
    not_completed_experiments = 0
    completed_experiments = 0
    
    for cfg in config_files:
        if(cfg["completed"]==True):
            completed_config_files.append(cfg)
            completed_experiments += 1
        else:
            not_completed_config_files.append(cfg)
            not_completed_experiments += 1
    # if(not_completed_experiments>0):
    # print(f"Completed experiments: {completed_experiments} \nNot completed experiments: {not_completed_experiments}")
    total_experiments = completed_experiments + not_completed_experiments
    if(log):
        print(f"Completed experiments: {completed_experiments}/{total_experiments}")
    return completed_config_files,not_completed_config_files

def load_json_dict(path):
    with open(path) as f:
        file = json.load(f)
    return file

def load_dict_from_paths(paths, file_name):
    # Load dicts from paths
    dict_list = []
    for path in paths:
        full_file_name = os.path.join(path,file_name)
        mydict = load_json_dict(full_file_name)
        dict_list.append(mydict)
    return dict_list

def load_numpy_from_paths(paths,file_name):
    # Load numpy file from paths
    np_list = []
    for path in paths:
        full_file_name = os.path.join(path,file_name)
        np_item = np.load(full_file_name)
        np_list.append(np_item)
    return np_list
    
def load_json_from_exp(paths, file_name):
    # Load json file from paths
    roc_list = []
    for path in paths:
        full_file_name = os.path.join(path,file_name)
        roc_dict = json.load(open(full_file_name,"r")) 
        roc_list.append(roc_dict)
    return roc_list

def load_pickle(path):
    with open(path, 'rb') as file:
        return pickle.load(file)
        
def extract_key_from_histories(history_list, max_len=1000, key="loss"):
    num_items = len(history_list)
    history_matrix = np.NaN * np.ones((num_items,max_len))
    for i,h_dict in enumerate(history_list):
        h = h_dict[key]
        len_h = len(h)
        history_matrix[i,:len_h] = h
    return history_matrix

def extract_auroc_from_roc_list(roc_list):
    num_items = len(roc_list)
    auroc_vector = np.zeros((num_items))
    for i,roc_dict in enumerate(roc_list):
        auroc_vector[i] = roc_dict["roc_auc"]
    return auroc_vector

def save_png_eps_figure(filename):
    full_filename = os.path.join(current_path,"images",filename)
    for extension in [".png",".eps",".pdf"]:
        plt.savefig(full_filename+extension,bbox_inches='tight')

def get_all_experiments_paths(base_path, model, dataset, optimizer, learning_rate, depth=1):
    # Function to get all the result paths for a given model, dataset, split, optimizer, learning rate
    lr = str(learning_rate)
    folder_path = os.path.join(base_path,f"{model}_{dataset}",f"opt_{optimizer}_lr_{lr}")
    print(folder_path)
    return get_subfolders_at_depth(folder_path=folder_path,depth=depth)

learning_rates=[1e-3, 5e-4, 1e-4, 5e-5, 1e-5, 5e-6, 1e-6] #(1e-4 1e-5 1e-6) #
reconstruction_signal_weights=[1.0, 0.1, 0.01]
# seeds=(42) #seeds=(42, 123, 456, 789, 1011)  # Define the seeds
seeds=[42, 123, 456, 789, 1011]
exp_paths_inria_vae = []
exp_paths_latent_lead = []
for lr in learning_rates:
    exp_paths_inria_vae.extend(get_all_experiments_paths(base_path=results_path,model="vae-inria",dataset="inria",
                            optimizer="adam", learning_rate=lr))
    exp_paths_latent_lead.extend(get_all_experiments_paths(base_path=results_path,model="latent-lead",dataset="inria_gat",
                            optimizer="adam", learning_rate=lr))
inria_vae_conf_files = get_conf_from_paths(exp_paths_inria_vae)
inria_vae_completed_conf_files, inria_vae_incompleted_conf_files = get_completed_conf_files(inria_vae_conf_files)
latent_lead_conf_files = get_conf_from_paths(exp_paths_latent_lead)
latent_lead_completed_conf_files, latent_lead_incompleted_conf_files = get_completed_conf_files(latent_lead_conf_files)

In [ ]:
def get_latent_lead_losses_with_removed_leads(exp_path):
    removed_leads_range = range(0, 250, 20)

    train_losses_dict = {
        "loss": [],
        "acti_loss": [],
        "signal_loss": [],
        "mae_acti_loss": []
    }

    test_losses_dict = {
        "loss": [],
        "acti_loss": [],
        "signal_loss": [],
        "mae_acti_loss": []
    }

    val_losses_dict = {
        "loss": [],
        "acti_loss": [],
        "signal_loss": [],
        "mae_acti_loss": []
    }

    for num_removed_leads in removed_leads_range:
        for slice in ["test", "val"]: #["train", "test", "val"]
            loss_dict = load_json_dict(os.path.join(exp_path, f"{slice}_loss_{num_removed_leads}"))
            if slice == "train":
                for key in train_losses_dict:
                    train_losses_dict[key].append(loss_dict[key])
            elif slice == "test":
                for key in test_losses_dict:
                    test_losses_dict[key].append(loss_dict[key])
            elif slice == "val":
                for key in val_losses_dict:
                    val_losses_dict[key].append(loss_dict[key])
    # Convert lists to numpy arrays
    train_losses_dict = {key: np.array(value) for key, value in train_losses_dict.items()}
    test_losses_dict = {key: np.array(value) for key, value in test_losses_dict.items()}
    val_losses_dict = {key: np.array(value) for key, value in val_losses_dict.items()}

    return train_losses_dict, test_losses_dict, val_losses_dict

In [ ]:
def get_loss_dicts_from_path(exp_path):
    train_loss_dict = load_json_dict(os.path.join(exp_path, f"train_loss"))
    test_loss_dict = load_json_dict(os.path.join(exp_path, f"test_loss"))
    val_loss_dict = load_json_dict(os.path.join(exp_path, f"val_loss"))

    return train_loss_dict, test_loss_dict, val_loss_dict

In [ ]:
def get_avg_std_test_loss_from_path(exp_path, key):
    test_loss_dict = load_pickle(os.path.join(exp_path, f"test_loss_stats"))
    test_loss = test_loss_dict[key]
    test_loss_avg = test_loss["avg"]
    test_loss_std = test_loss["std"]
    return test_loss_avg,test_loss_std

In [ ]:
# This computes the onset position as the distance between the the cluster centers
# This info is stored inside onset_dict_list{0,50,...}, which contain the recorded cluster position
# for all the samples in the test set
def calculate_average_distance(records):
    if not records:
        return 0
    total_distance = sum(item['avg_distance_clusters'] for item in records if 'avg_distance_clusters' in item)
    return total_distance / len(records)

def calculate_standard_deviation(records):
    if not records:
        return 0
    avg_distance = calculate_average_distance(records)
    variance = sum((item['avg_distance_clusters'] - avg_distance) ** 2 for item in records if 'avg_distance_clusters' in item) / len(records)
    return math.sqrt(variance)

def get_onset_position_errors(exp_path, removed_leads_range):
    # removed_leads_range = range(0, 250, 50)
    
    onset_errors_dict = {
        "avg_d_min": [],
        "std_d_min": [],   
    }

    for num_removed_leads in removed_leads_range:
        onset_dict_list = load_pickle(os.path.join(exp_path, f"onsets_dict_list_{num_removed_leads}"))
        avg_onset_dist = calculate_average_distance(onset_dict_list)
        std_dev_onset_dist = calculate_standard_deviation(onset_dict_list)
        onset_errors_dict["avg_d_min"].append(avg_onset_dist)
        onset_errors_dict["std_d_min"].append(std_dev_onset_dist)

    # Convert lists to numpy arrays
    onset_errors_dict = {key: np.array(value) for key, value in onset_errors_dict.items()}

    return onset_errors_dict

In [ ]:
# # This computes the onset position as the distance between the closest surface point to the cluster centers
# # This info is stored inside onset_d_dict_{0,50,...}
# def get_onset_position_errors(exp_path, removed_leads_range):
#     # removed_leads_range = range(0, 250, 50)
    
#     onset_errors_dict = {
#         "avg_d_min": [],
#         "std_d_min": [],    
#     }

#     for num_removed_leads in removed_leads_range:
#         loss_dict = load_json_dict(os.path.join(exp_path, f"onset_d_dict_{num_removed_leads}"))
#         for key in onset_errors_dict:
#             onset_errors_dict[key].append(loss_dict[key])

#     # Convert lists to numpy arrays
#     onset_errors_dict = {key: np.array(value) for key, value in onset_errors_dict.items()}

#     return onset_errors_dict

In [ ]:
# Plot one example loss vs removed leads for latent-lead 
removed_leads_range = np.arange(0,250,20)
example_path_latent_lead = "/latent-lead/example/path" # Change it to your example
_, test_losses_dict_latent_lead, val_losses_dict_latent_lead = get_latent_lead_losses_with_removed_leads(example_path_latent_lead)
test_acti_loss_array_latent_lead = test_losses_dict_latent_lead["acti_loss"]
# Get example test loss from vae_inria
example_path_inria_vae = "/vae_results/example/path" 
_, test_losses_dict_inria_vae, _ = get_loss_dicts_from_path(example_path_inria_vae)
max_acti_loss_vae_inria = test_losses_dict_inria_vae["rec_loss"]
# Plot test reconstruction loss for cardiac activation map
plt.figure()
plt.plot(removed_leads_range, test_acti_loss_array_latent_lead, color = "blue", label="Our model")
plt.axhline(y=max_acti_loss_vae_inria, color='red', linestyle='--', linewidth=1, label="Baseline")
# plt.vlines(x=134, ymin=np.min(test_acti_loss_array_latent_lead)-0.0002, ymax=max_acti_loss_vae_inria, colors='red', linestyles='--')
plt.vlines(x=134, ymin=0.0, ymax=max_acti_loss_vae_inria, colors='red', linestyles='--')
plt.xlim([0,250])
plt.ylim([np.min(test_acti_loss_array_latent_lead)-0.0002, 0.0350])
plt.ylim([0.0, 0.0350])
plt.gca().tick_params(axis='x', which='both', direction='inout', length=6)
plt.gca().set_xticks(list(plt.gca().get_xticks()) + [134])
plt.xlabel("Number of removed leads", fontsize=labelssize)
plt.ylabel("Activation Loss", fontsize=labelssize)
plt.xticks(fontsize=ticksize)
plt.yticks(fontsize=ticksize)
plt.legend(fontsize=legendsize)
save_png_eps_figure("acti_loss_vs_removed_leads_and_baseline_start_0")

In [ ]:
latent_lead_result_dict_list = []
for conf in latent_lead_completed_conf_files:
    exp_path = conf["path"]
    lr = conf["optimizer"]["learning_rate"]
    reconstruction_signal_weight = conf["model"]["reconstruction_signal_weight"]
    alpha_loss_weight = conf["model"]["alpha_loss_weight"]
    md_name = conf["model"]["name"]
    ds_name = conf["dataset"]["name"]
    seed = conf["seed"]
    # Get training, validation, test loss values for multiple dropped leads
    train_losses_dict, test_losses_dict, val_losses_dict = get_latent_lead_losses_with_removed_leads(exp_path)
    # Get final training, validation, test loss values 
    train_loss, test_loss, val_loss  = get_loss_dicts_from_path(exp_path)
    # Get statistics of test loss activation error
    test_acti_loss_avg, test_acti_loss_std = get_avg_std_test_loss_from_path(exp_path,"acti_loss")
    test_mae_acti_loss_avg, test_mae_acti_loss_std = get_avg_std_test_loss_from_path(exp_path,"mae_acti_loss")
    # Get onset position errors for multiple dropped leads 
    onset_errors_dict = get_onset_position_errors(exp_path, removed_leads_range=[0,50,100,150,200])
    # Get model size
    model_size_dict = load_json_dict(os.path.join(exp_path,"model_size"))
    model_size_mb = model_size_dict["model_size_mb"] 
    num_parameters = model_size_dict["num_parameters"] 
    gnn_num_parameters = model_size_dict["gnn"]["num_parameters"] 
    fmmhead_num_parameters = model_size_dict["fmm_head"]["num_parameters"] 
    encoder_num_parameters = model_size_dict["encoder"]["num_parameters"] 
    decoder_num_parameters = model_size_dict["decoder"]["num_parameters"] 
    # Get train time
    train_time_dict = load_json_dict(os.path.join(exp_path,"train_time"))
    train_time = train_time_dict["train_time"]
    avg_epoch_train_time = np.average(train_time_dict["train_epochs_time"])
    num_epochs = len(train_time_dict["train_epochs_time"])
    dict_to_add = {"Dataset": ds_name, "Model": md_name, "Learning Rate":lr, "Seed":seed,
                   "reconstruction_signal_weight": reconstruction_signal_weight,
                   "alpha_loss_weight": alpha_loss_weight,
                   "model_num_parameters":num_parameters,
                   "model_size_mb": model_size_mb,
                   "gnn_num_parameters": gnn_num_parameters,
                   "fmmhead_num_parameters": fmmhead_num_parameters,
                   "encoder_num_parameters": encoder_num_parameters,
                   "decoder_num_parameters": decoder_num_parameters,
                   "train_time":train_time,
                   "avg_epoch_train_time": avg_epoch_train_time,
                   "num_epochs":num_epochs,
                    #  Train loss with removed leads
                    "train_loss_removed_leads": train_losses_dict["loss"],
                    "train_acti_loss_removed_leads": train_losses_dict["acti_loss"],
                    "train_signal_loss_removed_leads": train_losses_dict["signal_loss"],
                    "train_mae_acti_loss_removed_leads": train_losses_dict["mae_acti_loss"],
                    "train_loss_best_final_value": train_loss["loss"],
                    "train_acti_loss_best_final_value": train_loss["acti_loss"],
                    "train_signal_loss_best_final_value": train_loss["signal_loss"],
                    "train_mae_acti_loss_best_final_value": train_loss["mae_acti_loss"],

                    #  Val loss with removed leads
                    "val_loss_removed_leads": val_losses_dict["loss"],
                    "val_acti_loss_removed_leads": val_losses_dict["acti_loss"],
                    "val_signal_loss_removed_leads": val_losses_dict["signal_loss"],
                    "val_mae_acti_loss_removed_leads": val_losses_dict["mae_acti_loss"],
                    "val_loss_best_final_value": val_loss["loss"],
                    "val_acti_loss_best_final_value": val_loss["acti_loss"],
                    "val_signal_loss_best_final_value": val_loss["signal_loss"],
                    "val_mae_acti_loss_best_final_value": val_loss["mae_acti_loss"],

                    #  Test loss with removed leads
                    "test_loss_removed_leads": test_losses_dict["loss"],
                    "test_acti_loss_removed_leads": test_losses_dict["acti_loss"],
                    "test_signal_loss_removed_leads": test_losses_dict["signal_loss"],
                    "test_mae_acti_loss_removed_leads": test_losses_dict["mae_acti_loss"],
                    "test_loss_best_final_value": test_loss["loss"],
                    "test_acti_loss_best_final_value": test_loss["acti_loss"],
                    "test_signal_loss_best_final_value": test_loss["signal_loss"],
                    "test_mae_acti_loss_best_final_value": test_loss["mae_acti_loss"],

                    # Test loss statistics
                    "test_acti_loss_avg": test_acti_loss_avg,
                    "test_acti_loss_std": test_acti_loss_std,
                    "test_mae_acti_loss_avg":test_mae_acti_loss_avg,
                    "test_mae_acti_loss_std":test_mae_acti_loss_std,
                    
                    # Spatial distances
                    "onset_avg_d_min_removed_leads": onset_errors_dict["avg_d_min"],
                    "onset_std_d_min_removed_leads": onset_errors_dict["std_d_min"],
                    "onset_avg_d_min_best_final_value": onset_errors_dict["avg_d_min"][0],
                    "onset_std_d_min_best_final_value": onset_errors_dict["std_d_min"][0],
    }

    latent_lead_result_dict_list.append(dict_to_add)
# latent_lead_result_dict_list

In [ ]:
latent_lead_df = pd.DataFrame(latent_lead_result_dict_list)
latent_lead_df_test_loss = latent_lead_df[["Model","Learning Rate",
                                           "reconstruction_signal_weight",
                                           "alpha_loss_weight",
                                           "test_acti_loss_avg",
                                           "test_acti_loss_std",
                                           "test_mae_acti_loss_avg",
                                           "test_mae_acti_loss_std",
                                           "onset_avg_d_min_best_final_value",
                                           "onset_std_d_min_best_final_value",
                                           "Seed"]]
# Groupy hyperparameters and take only first experiment if there are multiple ones
grouped_latent_lead_df_test_loss = latent_lead_df_test_loss.groupby(["Learning Rate", "reconstruction_signal_weight", "alpha_loss_weight"]).first().reset_index()

best_model = grouped_latent_lead_df_test_loss.loc[grouped_latent_lead_df_test_loss["test_acti_loss_avg"].idxmin()]
best_learning_rate = best_model["Learning Rate"]
best_reconstruction_signal_weight = best_model["reconstruction_signal_weight"]

print(best_learning_rate, best_reconstruction_signal_weight)
grouped_latent_lead_df_test_loss

In [ ]:
vae_inria_result_dict_list = []
for conf in inria_vae_completed_conf_files:
    exp_path = conf["path"]
    lr = conf["optimizer"]["learning_rate"]
    md_name = conf["model"]["name"]
    ds_name = conf["dataset"]["name"]
    seed = conf["seed"]
    
    # Get training, validation, test loss values
    train_losses_dict, test_losses_dict, val_losses_dict = get_loss_dicts_from_path(exp_path)

    # Get statistics of test loss activation error
    test_acti_loss_avg, test_acti_loss_std = get_avg_std_test_loss_from_path(exp_path,"rec_loss")
    test_mae_acti_loss_avg, test_mae_acti_loss_std = get_avg_std_test_loss_from_path(exp_path,"rec_loss_mae")

    # Get onset position errors for multiple dropped leads 
    onset_errors_dict = get_onset_position_errors(exp_path, removed_leads_range=[0])
    
    # Get model size
    model_size_dict = load_json_dict(os.path.join(exp_path, "model_size"))
    model_size_mb = model_size_dict["model_size_mb"]
    num_parameters = model_size_dict["num_parameters"]
    
    # Get train time
    train_time_dict = load_json_dict(os.path.join(exp_path, "train_time"))
    train_time = train_time_dict["train_time"]
    avg_epoch_train_time = np.average(train_time_dict["train_epochs_time"])
    num_epochs = len(train_time_dict["train_epochs_time"])

    dict_to_add = {
        "Dataset": ds_name,
        "Model": md_name,
        "Learning Rate": lr,
        "Seed": seed,

        "model_num_parameters": num_parameters,
        "model_size_mb": model_size_mb,
        "train_time": train_time,
        "avg_epoch_train_time": avg_epoch_train_time,
        "num_epochs": num_epochs,

        # Train loss 
        "train_loss": train_losses_dict["loss"],
        "train_rec_loss": train_losses_dict["rec_loss"],
        "train_rec_loss_gaus": train_losses_dict["rec_loss_gaus"],
        "train_kl_loss": train_losses_dict["kl_loss"],
        "train_rec_loss_mae": train_losses_dict["rec_loss_mae"],

        # Val loss 
        "val_loss": val_losses_dict["loss"],
        "val_rec_loss": val_losses_dict["rec_loss"],
        "val_rec_loss_gaus": val_losses_dict["rec_loss_gaus"],
        "val_kl_loss": val_losses_dict["kl_loss"],
        "val_rec_loss_mae": val_losses_dict["rec_loss_mae"],

        # Test loss 
        "test_loss": test_losses_dict["loss"],
        "test_rec_loss": test_losses_dict["rec_loss"],
        "test_rec_loss_gaus": test_losses_dict["rec_loss_gaus"],
        "test_kl_loss": test_losses_dict["kl_loss"],
        "test_rec_loss_mae": test_losses_dict["rec_loss_mae"],

        # Test loss statistics
        "test_acti_loss_avg": test_acti_loss_avg,
        "test_acti_loss_std": test_acti_loss_std,
        "test_mae_acti_loss_avg":test_mae_acti_loss_avg,
        "test_mae_acti_loss_std":test_mae_acti_loss_std,

        # Spatial distances
        "onset_avg_d_min_best_final_value": onset_errors_dict["avg_d_min"][0],
        "onset_std_d_min_best_final_value": onset_errors_dict["std_d_min"][0],
    }

    vae_inria_result_dict_list.append(dict_to_add)


In [ ]:
inria_vae_df = pd.DataFrame(vae_inria_result_dict_list)
inria_vae_df_test_loss = inria_vae_df[["Model","Learning Rate",
                                       "test_rec_loss",
                                       "test_rec_loss_mae",
                                       "test_acti_loss_avg",
                                       "test_acti_loss_std",
                                       "test_mae_acti_loss_avg",
                                       "test_mae_acti_loss_std",
                                       "onset_avg_d_min_best_final_value",
                                       "onset_std_d_min_best_final_value",
                                       "Seed"]]
grouped_inria_vae_df_test_loss = inria_vae_df_test_loss.groupby(["Learning Rate"]).first().reset_index()

best_model = grouped_inria_vae_df_test_loss.loc[grouped_inria_vae_df_test_loss["test_rec_loss"].idxmin()]
best_learning_rate = best_model["Learning Rate"]

print(best_learning_rate)
grouped_inria_vae_df_test_loss